In [2]:
import pennylane as qml
from pennylane import numpy as np


def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])
        
def substract_m_n(wires_m, wires_n):
    # |m>|n> ->  |m - n> |n>
    
    qml.QFT(wires = wires_m)
    
    
    for i in range(len(wires_n)):
        qml.ctrl(add_k_fourier, control=wires_n[i])(-2 **(len(wires_n) - i - 1), wires_m)
        
    qml.adjoint(qml.QFT)(wires = wires_m)

dev = qml.device("default.qubit", wires = 6, shots = 1)

@qml.qnode(dev)
def circuit():
    qml.BasisEmbedding(4, wires = range(3))
    qml.BasisEmbedding(2, wires = range(3,6))
    substract_m_n(range(3), range(3,6))
    return qml.sample(wires = range(3))

circuit()

tensor([0, 1, 0], requires_grad=True)

In [3]:
def is_greater(wires_m, wires_n, wires_aux, wires_target):
    
    wires_m_aux = [wires_aux] + list(wires_m)
    substract_m_n(wires_m_aux, wires_n)
    
    qml.PauliX(wires = wires_aux)
    qml.CNOT(wires = [wires_aux, wires_target])
    qml.PauliX(wires = wires_aux)
    
    qml.adjoint(substract_m_n)(wires_m_aux, wires_n)
    
dev = qml.device("default.qubit", wires = 8, shots = 1)

@qml.qnode(dev)
def circuit():
    qml.BasisEmbedding(3, wires = range(3))
    qml.BasisEmbedding(3, wires = range(3,6))
    is_greater(range(3), range(3,6), 6, 7)
    return qml.sample(wires = 7)

circuit()

tensor(1, requires_grad=True)

In [4]:
wires_dividendo = [0,1,2,3,4,5]
wires_divisor = [6,7,8,9]
wires_aux_dividendo = [10,11,12]
wires_solucion = [13,14,15,16,17,18]
wires_aux = 19

def division(wires_dividendo, wires_divisor, wires_aux_dividendo, wires_solucion, wires_aux):
    
    wires_dividendo_totales = wires_aux_dividendo + wires_dividendo
    
    for i in range(len(wires_solucion)):
        window = wires_dividendo_totales[i - 1: len(wires_divisor)+ i]
        if i == 0:
            window = wires_dividendo_totales[0: len(wires_divisor)]
        is_greater(window, wires_divisor, wires_aux, wires_solucion[i])
        qml.ctrl(substract_m_n, control = wires_solucion[i])(window, wires_divisor)
        
        
        
dev = qml.device("default.qubit", wires = 20, shots = 1)

@qml.qnode(dev)
def circuit():
    qml.BasisEmbedding(32, wires = wires_dividendo)
    qml.BasisEmbedding(13, wires = wires_divisor)
    division(wires_dividendo, wires_divisor, wires_aux_dividendo, wires_solucion, wires_aux)
    
    
    return qml.sample(wires = wires_dividendo + wires_divisor + wires_solucion)

circuit()

tensor([0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0], requires_grad=True)